# Lab 5

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-510/blob/main/book/labs/lab_05.ipynb)

## Exercise 1: Calculating Distances with Functions

- Define a function `calculate_distance` that takes two geographic coordinates (latitude and longitude) and returns the distance between them using the Haversine formula.
- Use this function to calculate the distance between multiple pairs of coordinates.

In [ ]:
# Haversine function
import numpy as np


def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (
        np.sin(dlat / 2) ** 2
        + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    )
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance


# Usage : distances between multiple pairs of coordinates
coords = [
    (52.2296756, 21.0122287, 41.8919300, 12.5113300),
    (34.052235, -118.243683, 36.169941, -115.139832),
    (48.856613, 2.352222, 51.507351, -0.127758),
]

for coord in coords:
    lat1, lon1, lat2, lon2 = coord
    distance = calculate_distance(lat1, lon1, lat2, lon2)
    print(f"Distance: {distance} km")

## Exercise 2: Batch Distance Calculation

- Create a function `batch_distance_calculation` that accepts a list of coordinate pairs and returns a list of distances between consecutive pairs.
- Test the function with a list of coordinates representing several cities.

In [ ]:
def batch_distance_calculation(coord_list):
    distances = []
    for i in range(len(coord_list) - 1):
        lat1, lon1 = coord_list[i]
        lat2, lon2 = coord_list[i + 1]
        distance = calculate_distance(lat1, lon1, lat2, lon2)
        distances.append(distance)
    return distances


# Example usage
coordinates = [(35.6895, 139.6917), (34.0522, -118.2437), (40.7128, -74.0060)]
distances = batch_distance_calculation(coordinates)
print(f"Distances: {distances}")

## Exercise 3: Creating and Using a Point Class

- Define a `Point` class to represent a geographic point with attributes `latitude`, `longitude`, and `name`.
- Add a method `distance_to` that calculates the distance from one point to another.
- Instantiate several `Point` objects and calculate the distance between them.

In [ ]:
class Point:
    def __init__(self, latitude, longitude, name=None):
        self.latitude = latitude
        self.longitude = longitude
        self.name = name

    def __str__(self):
        return f"{self.name or 'Point'} ({self.latitude}, {self.longitude})"

    def distance_to(self, other_point):
        return calculate_distance(
            self.latitude, self.longitude, other_point.latitude, other_point.longitude
        )


# Defining some points representing some cities
city1 = Point(35.6895, 139.6917, "Tokyo")
city2 = Point(34.0522, -118.2437, "Los Angeles")
city3 = Point(40.7128, -74.0060, "New York")
# Calculating distances between cities
distance_tokyo_la = city1.distance_to(city2)
distance_la_ny = city2.distance_to(city3)
distance_ny_tokyo = city3.distance_to(city1)

## Exercise 4: Reading and Writing Files

- Write a function `read_coordinates` that reads a file containing a list of coordinates (latitude, longitude) and returns them as a list of tuples.
- Write another function `write_coordinates` that takes a list of coordinates and writes them to a new file.
- Ensure that both functions handle exceptions, such as missing files or improperly formatted data.

In [ ]:
# Q1
def read_coordinates(file_path):
    try:
        with open(file_path, "r") as file:
            coordinates = []
            for line in file:
                try:
                    lat, lon = line.strip().split(",")
                    coordinates.append((float(lat), float(lon)))
                except ValueError:
                    print(f"Skipping malformed line: {line.strip()}")
            return coordinates
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

In [ ]:
# Q2
def write_coordinates(coordinates, file_path):
    try:
        with open(file_path, "w") as file:
            for coord in coordinates:
                try:
                    file.write(f"{coord[0]},{coord[1]}\n")
                except Exception as e:
                    print(f"Error writing coordinate {coord}: {e}")
    except IOError as e:
        print(f"Error writing to file {file_path}: {e}")

## Exercise 5: Processing Coordinates from a File

- Create a function that reads coordinates from a file and uses the `Point` class to create `Point` objects.
- Calculate the distance between each consecutive pair of points and write the results to a new file.
- Ensure the function handles file-related exceptions and gracefully handles improperly formatted lines.

In [ ]:
# Create a sample coordinates.txt file
sample_data = """35.6895,139.6917
34.0522,-118.2437
51.5074,-0.1278
-33.8688,151.2093
48.8566,2.3522"""

output_file = "coordinates.txt"

try:
    with open(output_file, "w") as file:
        file.write(sample_data)
    print(f"Sample file '{output_file}' has been created successfully.")
except Exception as e:
    print(f"An error occurred while creating the file: {e}")

In [ ]:
def create_distances(file_path):
    points_list = read_coordinates(file_path)
    points_objects = [Point(lat, lon) for lat, lon in points_list]
    distances = []
    for i in range(len(points_objects) - 1):
        distances.append(points_objects[i].distance_to(points_objects[i + 1]))
    # writing distances to a file
    try:
        with open("distances.txt", "w") as file:
            for d in distances:
                try:
                    file.write(f"{d}\n")
                except Exception as e:
                    print(f"Error writing distance {d}: {e}")
    except IOError as e:
        print(f"Error writing to file distances.txt: {e}")

## Exercise 6: Exception Handling in Data Processing

- Modify the `batch_distance_calculation` function to handle exceptions that might occur during the calculation, such as invalid coordinates.
- Ensure the function skips invalid data and continues processing the remaining data.

In [ ]:
# Exception handling in batch distance calculation
def batch_distance_calculation(coord_list):
    distances = []
    for i in range(len(coord_list) - 1):
        coords1 = coord_list[i].strip().split(",")
        coords2 = coord_list[i + 1].strip().split(",")
        if len(coords1) != 2 or len(coords2) != 2:
            continue
        try:
            lat1, lon1 = coord_list[i]
            lat2, lon2 = coord_list[i + 1]
            distance = calculate_distance(lat1, lon1, lat2, lon2)
            distances.append(distance)
        except Exception as e:
            print(f"Error calculating distance between points {i} and {i + 1}: {e}")
    return distances

## Exercise 7: NumPy Array Operations and Geospatial Coordinates

In this exercise, you will work with NumPy arrays representing geospatial coordinates (latitude and longitude) and perform basic array operations.

1. Create a 2D NumPy array containing the latitude and longitude of the following cities: Tokyo (35.6895, 139.6917), New York (40.7128, -74.0060), London (51.5074, -0.1278), and Paris (48.8566, 2.3522).
2. Convert the latitude and longitude values from degrees to radians using np.radians().
3. Calculate the element-wise difference between Tokyo and the other cities' latitude and longitude in radians.

In [ ]:
import numpy as np

# Creating a DataFrame
coords = np.array(
    [
        [35.6895, 139.6917],  # Tokyo
        [34.0522, -118.2437],  # Los Angeles
        [51.5074, -0.1278],  # London
    ]
)
rad_coords = np.radians(coords)  # Convert to radians

# Calculate element-wise differences between Tokyo and other cities
diff = rad_coords - rad_coords[0]
print(diff)

## Exercise 8: Pandas DataFrame Operations with Geospatial Data

In this exercise, you'll use Pandas to load and manipulate a dataset containing city population data, and then calculate and visualize statistics.

1. Load the world cities dataset from this URL using Pandas: https://github.com/opengeos/datasets/releases/download/world/world_cities.csv
2. Display the first 5 rows and check for missing values.
3. Filter the dataset to only include cities with a population greater than 1 million.
4. Group the cities by their country and calculate the total population for each country.
5. Sort the cities by population in descending order and display the top 10 cities.

In [ ]:
import pandas as pd

url = "https://github.com/opengeos/datasets/releases/download/world/world_cities.csv"
df = pd.read_csv(url)
df.head()  # Display the first few rows of the DataFrame

In [ ]:
len(df)

In [ ]:
df = df[df["population"] > 1000000]

In [ ]:
len(df)  # Number of rows in the DataFrame

In [ ]:
df_grouped = df.groupby("country")["population"].sum().reset_index()
df_grouped.head()  # Display the first few rows of the grouped DataFrame

In [ ]:
sorted_df = df.sort_values(by="population", ascending=False)
sorted_df.head()  # Display the first few rows of the sorted DataFrame

## Exercise 9: Creating and Manipulating GeoDataFrames with GeoPandas

This exercise focuses on creating and manipulating GeoDataFrames, performing spatial operations, and visualizing the data.

1. Load the New York City building dataset from the GeoJSON file using GeoPandas: https://github.com/opengeos/datasets/releases/download/places/nyc_buildings.geojson
2. Create a plot of the building footprints and color them based on the building height (use the `height_MS` column).
3. Create an interactive map of the building footprints and color them based on the building height (use the `height_MS` column).
4. Calculate the average building height (use the `height_MS` column).
5. Select buildings with a height greater than the average height.
6. Save the GeoDataFrame to a new GeoJSON file.

In [ ]:
import geopandas as gpd

url = "https://github.com/opengeos/datasets/releases/download/places/nyc_buildings.geojson"
gdf = gpd.read_file(url)

In [ ]:
gdf  # Display the first few rows of the GeoDataFrame

In [ ]:
gdf.plot("height_MS", cmap="viridis", legend=True, figsize=(10, 10))

In [ ]:
gdf.explore("height_MS")

In [ ]:
# Average height of buildings
average_height = gdf["height_MS"].mean()
average_height

In [ ]:
# Buildings higher than average
taller_buildings = gdf[gdf["height_MS"] > average_height]
taller_buildings

In [ ]:
# Save the taller df to a geojson file
taller_buildings.to_file("taller_buildings.geojson", driver="GeoJSON")

## Exercise 10: Combining NumPy, Pandas, and GeoPandas

This exercise requires you to combine the power of NumPy, Pandas, and GeoPandas to analyze and visualize spatial data.

1. Use Pandas to load the world cities dataset from this URL: https://github.com/opengeos/datasets/releases/download/world/world_cities.csv
2. Filter the dataset to include only cities with latitude values between -40 and 60 (i.e., cities located in the Northern Hemisphere or near the equator).
3. Create a GeoDataFrame from the filtered dataset by converting the latitude and longitude into geometries.
4. Reproject the GeoDataFrame to the Mercator projection (EPSG:3857).
5. Calculate the distance (in meters) between each city and the city of Paris.
6. Plot the cities on a world map, coloring the points by their distance from Paris.

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/world/world_cities.csv"
world_cities = pd.read_csv(url)

In [ ]:
# Northern Hemisphere cities
northern_cities = world_cities[
    (world_cities["latitude"] > -40) & (world_cities["latitude"] < 60)
]
northern_cities

In [ ]:
gdf_cities = gpd.GeoDataFrame(
    northern_cities,
    geometry=gpd.points_from_xy(northern_cities.longitude, northern_cities.latitude),
)
gdf_cities.head()  # Display the first few rows of the GeoDataFrame with cities

In [ ]:
# distance from each city to paris
paris = gdf_cities[gdf_cities["name"] == "Paris"].geometry.values[0]
gdf_cities["distance_to_paris"] = gdf_cities.geometry.distance(paris)
gdf_cities[["name", "distance_to_paris"]].head()

In [ ]:
gdf_cities.plot("distance_to_paris", cmap="coolwarm", legend=False, figsize=(10, 10))

## Submission Requirements

Complete the exercises above and and upload the notebook to your GitHub repository. Make sure the notebook has a Colab badge at the top so that it can be easily opened in Google Colab. Submit the URL of the notebook to Canvas.